In [4]:
# Используем вкладку "Секреты" левой панели Colab для хранения ключей
from google.colab import userdata
client_id = userdata.get('SBER_ID')
auth = userdata.get('SBER_AUTH')

In [5]:
import requests
import uuid

def get_token(auth_token, scope='GIGACHAT_API_PERS'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «GIGACHAT_API_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    # Тело запроса
    payload = {
        'scope': scope
    }

    try:
        # Делаем POST запрос с отключенной SSL верификацией
        # (можно скачать сертификаты Минцифры, тогда отключать проверку не надо)
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return -1

In [6]:
response = get_token(auth)
if response != 1:
  print(response.text)
  giga_token = response.json()['access_token']

{"access_token":"eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.lwSsJ5yGNe5iC5w39jTry3-YICTEoeX9sQGTe22jym7vI_kvWaKAUxokViy3slSQvjWMBcJLdF8dyGgCh2HkIRKhEF1TrJ_5BKPiiiotpaitIjOXd_CWPVBOJeUTl5BzpRJ2Bwp1TtofjfFzHnYOeqP0LNQMgQ3IZN9lfTimhtRkcOURfrQ3N7EaWZXyMPtFBEUQV8TdDnYpCQE-B4FHiDRd9vZOkUYbQO-iu2DP-N9eymLVvcp53ivhXCqh9txvlQCZFFBJIFTCdf8waZcKbLsmJOa6vbP1lOCzzM4VyxfGH5P19X_lq6Bc42qhfDFeXsizRZGiGYXfFKLcUy60Fw.Y6BiNQZjbzhNaJWyC1r5GQ.6g_Zmrf__ZPwh0kxHCR2i2RPrlwOADmKkkXUrrvgLLMCGBvR4ix3NDX3pAb4CCoJIcYEFAvsu48giGLQuXSPW2luT8nMkOX-fUv3oUTN1gtFpkQeMjOVo65HhdsmfmAznZDily6lRM0PQ4rX5WEfXwoRSOpLneXe6tQVa3I7gJuXdmqgUib0R_oQGdOwerT1496sTZK1dVmtnz2RJgfmmzr5zEQbrs6Y1eUm6jKK6uHn3SBTVKPm6n8MSjLtauS0lkCVqSyaFrPrr1i5XuIEbpGtyugNVhVGhG1mHGMLhEwhq_qwUrI-EDtUerPnb35PM1V_RvjoLGqVlae-R5IiuRjbmmWRGkwJi0A-DkGMAc9YP-eRLeZsQprORkLW1dvjRMe49Ick1V0gVzi7Cbyr_vA6yazSBWmFM4p4sQ63asP_RZoYBgtTCoB7deO41V5S2kHgiORDkJzLQG-4mAgbpPLTACk41njcaswujlXvOMkwQ_pkBg4o2t20XWOHTSc0IKmfNLU2YB1lw3JeswPJabJiHT6jl

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
import requests

url = "https://gigachat.devices.sberbank.ru/api/v1/models"

payload={}
headers = {
  'Accept': 'application/json',
  'Authorization': f'Bearer {giga_token}'
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

print(response.text)

{"object":"list","data":[{"id":"GigaChat","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-2","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-2-Max","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-2-Pro","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Max","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Max-preview","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Plus","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Pro","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"GigaChat-Pro-preview","object":"model","owned_by":"salutedevices","type":"chat"},{"id":"Embeddings","object":"model","owned_by":"salutedevices","type":"embedder"},{"id":"Embeddings-2","object":"model","owned_by":"salutedevices","type":"embedder"},{"id":"EmbeddingsGigaR","object":"model","owned_by":"salutedevices","type"

In [8]:
import requests
import json


def get_chat_completion(auth_token, user_message):
    """
    Отправляет POST-запрос к API чата для получения ответа от модели GigaChat.

    Параметры:
    - auth_token (str): Токен для авторизации в API.
    - user_message (str): Сообщение от пользователя, для которого нужно получить ответ.

    Возвращает:
    - str: Ответ от API в виде текстовой строки.
    """
    # URL API, к которому мы обращаемся
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    # Подготовка данных запроса в формате JSON
    payload = json.dumps({
        "model": "GigaChat",  # Используемая модель
        "messages": [
            {
                "role": "user",  # Роль отправителя (пользователь)
                "content": user_message  # Содержание сообщения
            }
        ],
        "temperature": 1,  # Температура генерации
        "top_p": 0.1,  # Параметр top_p для контроля разнообразия ответов
        "n": 1,  # Количество возвращаемых ответов
        "stream": False,  # Потоковая ли передача ответов
        "max_tokens": 512,  # Максимальное количество токенов в ответе
        "repetition_penalty": 1,  # Штраф за повторения
        "update_interval": 0  # Интервал обновления (для потоковой передачи)
    })

    # Заголовки запроса
    headers = {
        'Content-Type': 'application/json',  # Тип содержимого - JSON
        'Accept': 'application/json',  # Принимаем ответ в формате JSON
        'Authorization': f'Bearer {auth_token}'  # Токен авторизации
    }

    # Выполнение POST-запроса и возвращение ответа
    try:
        response = requests.request("POST", url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        # Обработка исключения в случае ошибки запроса
        print(f"Произошла ошибка: {str(e)}")
        return -1

In [ ]:
# Автоматизация работы с данными
import pandas as pd
import requests
import json
from datetime import datetime

# Пример 1: Работа с pandas и обработка данных
def create_sample_dataframe():
    """Создание тестового DataFrame для демонстрации работы с pandas"""
    data = {
        'id': [1, 2, 3, 4, 5],
        'name': ['Алексей', 'Мария', 'Иван', 'Ольга', 'Дмитрий'],
        'age': [25, 30, 35, 28, 32],
        'city': ['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань'],
        'salary': [50000, 60000, 70000, 55000, 65000]
    }
    df = pd.DataFrame(data)
    return df

# Создаем и анализируем DataFrame
df = create_sample_dataframe()
print("Исходные данные:")
print(df)
print("\nОсновная статистика:")
print(df.describe())
print(f"\nСредняя зарплата: {df['salary'].mean():.2f}")

# Пример 2: Работа с внешним API и обработка JSON
def get_public_api_data():
    """Получение данных с публичного API и обработка JSON"""
    try:
        # Используем JSONPlaceholder API для демонстрации
        response = requests.get('https://jsonplaceholder.typicode.com/users', verify=False)
        if response.status_code == 200:
            users_data = response.json()

            # Обрабатываем JSON данные
            processed_data = []
            for user in users_data:
                processed_data.append({
                    'id': user['id'],
                    'name': user['name'],
                    'email': user['email'],
                    'city': user['address']['city'],
                    'company': user['company']['name']
                })

            # Создаем DataFrame из обработанных данных
            users_df = pd.DataFrame(processed_data)
            return users_df
        else:
            print(f"Ошибка API: {response.status_code}")
            return None
    except Exception as e:
        print(f"Ошибка при получении данных: {e}")
        return None

# Получаем и обрабатываем данные с API
print("\n" + "="*50)
print("Данные с внешнего API:")
users_df = get_public_api_data()
if users_df is not None:
    print(users_df.head())

# Пример 3: Интеграция с GigaChat для анализа данных
def analyze_data_with_gigachat(auth_token, data_description, question):
    """Использование GigaChat для анализа данных"""
    prompt = f"""
    У меня есть набор данных: {data_description}

    Вопрос: {question}

    Пожалуйста, предоставь краткий анализ и рекомендации на основе этих данных.
    """

    response = get_chat_completion(auth_token, prompt)
    if response != -1:
        return response.json()['choices'][0]['message']['content']
    else:
        return "Ошибка при обращении к GigaChat"

# Пример использования GigaChat для анализа
print("\n" + "="*50)
print("Анализ данных с помощью GigaChat:")

# Описание данных для анализа
data_description = f"""
DataFrame с {len(df)} строками и {len(df.columns)} колонками.
Колонки: {list(df.columns)}
Статистика по зарплатам: средняя {df['salary'].mean():.2f}, максимум {df['salary'].max()}
"""

analysis_result = analyze_data_with_gigachat(
    giga_token,
    data_description,
    "Какие insights можно извлечь из этих данных? Есть ли корреляция между возрастом и зарплатой?"
)

print(analysis_result)

# Пример 4: Создание отчета с использованием всех инструментов
def generate_comprehensive_report(auth_token, dataframe, api_data):
    """Генерация комплексного отчета с использованием pandas, API и GigaChat"""

    # Анализ данных с pandas
    pandas_analysis = {
        'total_records': len(dataframe),
        'columns': list(dataframe.columns),
        'numeric_stats': dataframe.describe().to_dict(),
        'categorical_summary': dataframe.select_dtypes(include=['object']).describe().to_dict()
    }

    # Подготовка отчета
    report_prompt = f"""
    Сгенерируй профессиональный отчет на основе анализа данных:

    1. Локальные данные (DataFrame):
    - Количество записей: {pandas_analysis['total_records']}
    - Колонки: {pandas_analysis['columns']}
    - Статистика: {json.dumps(pandas_analysis['numeric_stats'], ensure_ascii=False)}

    2. Внешние данные (API):
    - Получено {len(api_data) if api_data is not None else 0} записей пользователей

    Сделай выводы и предложи рекомендации для дальнейшего анализа.
    """

    giga_report = get_chat_completion(auth_token, report_prompt)
    if giga_report != -1:
        return giga_report.json()['choices'][0]['message']['content']
    else:
        return "Ошибка при генерации отчета"

# Генерация финального отчета
print("\n" + "="*50)
print("КОМПЛЕКСНЫЙ ОТЧЕТ:")
final_report = generate_comprehensive_report(giga_token, df, users_df)
print(final_report)

# Пример 5: Сохранение результатов в файлы
def save_results(dataframe, analysis_text, filename_prefix):
    """Сохранение результатов анализа в различные форматы"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Сохраняем DataFrame в CSV
    csv_filename = f"{filename_prefix}_{timestamp}.csv"
    dataframe.to_csv(csv_filename, index=False, encoding='utf-8')

    # Сохраняем анализ в текстовый файл
    txt_filename = f"{filename_prefix}_analysis_{timestamp}.txt"
    with open(txt_filename, 'w', encoding='utf-8') as f:
        f.write(analysis_text)

    # Сохраняем в JSON
    json_data = {
        'metadata': {
            'generated_at': timestamp,
            'records_count': len(dataframe),
            'columns': list(dataframe.columns)
        },
        'analysis': analysis_text
    }

    json_filename = f"{filename_prefix}_{timestamp}.json"
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=2)

    print(f"\nРезультаты сохранены в файлы:")
    print(f"- CSV: {csv_filename}")
    print(f"- TXT: {txt_filename}")
    print(f"- JSON: {json_filename}")

# Сохраняем результаты
save_results(df, final_report, "data_analysis_report")

print("\n" + "="*50)



Исходные данные:
   id     name  age             city  salary
0   1  Алексей   25           Москва   50000
1   2    Мария   30  Санкт-Петербург   60000
2   3     Иван   35      Новосибирск   70000
3   4    Ольга   28     Екатеринбург   55000
4   5  Дмитрий   32           Казань   65000

Основная статистика:
             id        age       salary
count  5.000000   5.000000      5.00000
mean   3.000000  30.000000  60000.00000
std    1.581139   3.807887   7905.69415
min    1.000000  25.000000  50000.00000
25%    2.000000  28.000000  55000.00000
50%    3.000000  30.000000  60000.00000
75%    4.000000  32.000000  65000.00000
max    5.000000  35.000000  70000.00000

Средняя зарплата: 60000.00

Данные с внешнего API:
   id              name                      email           city  \
0   1     Leanne Graham          Sincere@april.biz    Gwenborough   
1   2      Ervin Howell          Shanna@melissa.tv    Wisokyburgh   
2   3  Clementine Bauch         Nathan@yesenia.net  McKenziehaven   
3  